In [1]:
import os
from dotenv import load_dotenv
load_dotenv('../.env')
from pathlib import Path
PATH_WIND = Path(os.environ.get('PATH_WIND'))

In [ ]:
import geopandas as gpd
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
from typing import List

import warnings
warnings.filterwarnings('ignore')

## Analysis

We would like some basic statistics, for instance:
1. Comparisons between countries regarding output, area, number of turbines, if possible normalised with population size.
2. Temporal trends.
3. A detailed overview of company shares in wind farms. For this we would probably need Neo4J to query the ownership graph, or simplify this graph to a table. 
4. Perform linear regressions on costs of wind farms, because some data is lacking. 
5. Assess how much power is sold through PPAs

In [3]:
# Import windfarm shapes
wind = gpd.read_file(PATH_WIND.joinpath('gis', 'windfarms_v1.geojson'))
                     
# Import company data from Aleph
cw = pd.read_csv(PATH_WIND.joinpath('aleph', 'company_windfarm.csv'))

# Merge files
df = pd.merge(cw,
              wind[['mps_uuid', 'geometry']],
              on='mps_uuid',
              how='left')

# Convert to geopandas
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=4326)

### Stap 1: Nederland, bestaande windparken

Dit zijn de Nederlandse windparken die in bedrijf zijn.

In [6]:
nl_operational = wind[(wind.country=='nl') & (wind.status == 'OPERATIONAL')].copy()
nl_operational[['name', 'geometry', 'capacity_mw']].explore(tiles='CartoDB Positron')


## Stap 2: Nederland, met toekomstige plannen en parken in aanbouw

In [10]:
def plot_plans(df: pd.DataFrame,
               country: str,
               exclude: List[str]):
    
    df = df[(df.country==country) & (~df.status.isin(exclude))].copy()
    df['color'] = df.status.apply(lambda x: '#C62828' if x in ['OPERATIONAL'] else '#283593')
    Map = df[['name', 'status', 'capacity_mw', 'geometry']].explore(tiles='CartoDB Positron',
                                                                    color=df.color)
    capacity = df.capacity_mw.sum()
    
    return Map, [capacity]

In [11]:
exclude = ['AREA_UNDER_STATE_INVESTIGATION']

Map, stats = plot_plans(wind, 'nl', exclude)
Map

## Stap 3: Maar hoe zit het in andere landen?

### Noorwegen - niet zo ver

In [13]:
Map, stats = plot_plans(wind, 'no', exclude)
Map

### België evenmin

In [14]:
Map, stats = plot_plans(wind, 'be', exclude)
Map

### Maar Duitsland, met een hele kleine Noordzee, is een stuk verder

In [15]:
Map, stats = plot_plans(wind, 'de', exclude)
Map

### En Denemarken helemaal

In [16]:
Map, stats = plot_plans(wind, 'dk', exclude)
Map

### En het Verenigd Koninkrijk spant de kroon

In [17]:
Map, stats = plot_plans(wind, 'gb', exclude)
Map

## Groei van het vermogen

De bestaande en geplande hoeveelheid energie groeit de komende jaren enorm.

In [18]:
def plot_power(df: pd.DataFrame,     
               country: str,
               exclude: List[str],
               column: str = 'capacity_mw',
               cumulative: bool = True):
    
    df = df[~df.status.isin(exclude)]
    
    if country == 'all':
        df = df.groupby(['country','year'])[column].sum().reset_index()
        if cumulative:
            df = df.sort_values(by='year')
            df['cumsum'] = df[column].cumsum()
            return px.line(data_frame=df,
                           x='year',
                           y='cumsum',
                           title='Cumulative additional output per year for all countries'
                          )
        else:
            return px.bar(data_frame=df,
                           x='year',
                           y=[column],
                           color='country',
                           title='Additional capacity per year per country')
        
    else:
        df=df[df.country == country].copy()
        df = df.groupby('year')[column].sum().reset_index()
        if cumulative:
            df['cumsum'] = df[column].cumsum()
            return px.line(data_frame=df,
                           x='year',
                           y='cumsum',
                           title=f'Cumulative additional output per year for {country}')
        else:
            return px.bar(data_frame=df,
                          x='year',
                          y=[column],
                          title=f'Additonal capacity per year for {country}')
            

### Alle Noordzeelanden cumulatief

In 2030 vindt er een enorme sprong plaats. Dit is wel op voorwaarde dat alle plannen doorgaan en de beoogde capaciteit ook wordt gehaald.

In [21]:
fig = plot_power(df, 'all', exclude, 'capacity_mw', cumulative=True)
fig.add_vline(x='2023-01-01', line_width=1, line_dash='dash', line_color='red')

### Alle Noordzeelanden afzonderlijk

Het grootste deel van de groei komt voor rekening van het Verenigd Koninkrijk en Duitsland.

In [23]:
fig = px.line(df.groupby(['country','year']).capacity_mw.sum().groupby(level=0).cumsum().reset_index(),
              x='year',
              y='capacity_mw',
              color='country')

fig.add_vline(x='2023-01-01', 
              line_width=1, 
              line_dash='dash', 
              line_color='red') 

## Verandering over tijd

Je ziet pas echt hoeveel windparken er op zee bijkomen, als je de gebieden over tijd op een kaart zet.

## Vermogen per hoofd van de bevolking

In [24]:
pop = {'nl': 17800000,
       'dk': 5940000,
       'de': 84430000,
       'gb': 67020000,
       'no': 5500000,
       'be': 11690000}

In [27]:
df['pop'] = df.country.map(pop)
df['capacity_pop'] = df.capacity_mw / (df['pop'] / 1000000)

In [28]:
fig = px.line(df.groupby(['country','year']).capacity_pop.sum().groupby(level=0).cumsum().reset_index(),
              x='year',
              y='capacity_pop',
              color='country',
              title='Output of North Sea wind parks per country per million inhabitants')

fig.add_vline(x='2023-01-01', 
              line_width=1, 
              line_dash='dash', 
              line_color='red') 

<video width="600" height="450" 
       src="./timelapse_windfarms.mp4"  
       controls>
</video>

## Bedrijven

Ook Nederlandse bedrijven tellen niet echt mee, zelfs niet in onze eigen Noordzee.

In [36]:
selection = gdf.groupby(['jurisdiction', 'country', 'company']).agg({'output_company_mw_lower': 'sum',
                                                   'output_company_mw_upper': 'sum'}).reset_index()

In [37]:
fig = px.bar(selection.groupby(['jurisdiction'])\
           .agg({'output_company_mw_lower': 'sum',
                 'output_company_mw_upper': 'sum'})\
           .reset_index()\
           .sort_values(by='output_company_mw_lower', 
                        ascending=False),
           x='jurisdiction',
           y=['output_company_mw_lower', 'output_company_mw_upper'],
           barmode='group',
           title='Output of North Sea wind parks per company HQ jurisdiction',
           labels=dict(value='Output in MW', jurisdiction='Country HQ company'))

newnames = {'output_company_mw_lower': 'Lower limit MW', 'output_company_mw_upper': 'Upper limit MW'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))

fig.show()

In [38]:
fig = go.Figure(go.Parcats(
    dimensions=[
        {'label': 'country_company', 'values': selection.jurisdiction},
        {'label': 'country_asset', 'values': selection.country}
    ],
    counts=selection.output_company_mw_upper
                          ))

fig.show()